In [1]:
import random
import pickle
import random
import time

In [2]:
class QLearning:
    def __init__(self,epsilon=0.2,alpha=0.3,gamma=0.9):
        self.epsilon = epsilon
        self.alpha = alpha
        self.gamma = gamma
        self.Q = {}
        self.last_board = None
        self.q_last = 0.0
        self.state_action_last = None
        
        
    def game_begin(self):
        self.last_board = None
        self.q_last  = 0.0
        self.state_action_last = None
        
        
    def epsilon_greedy(self, state, possible_moves):
        self.last_board = tuple(state)
        if(random.random() < self.epsilon):
            move = random.choice(possible_moves)
            self.state_action_last = (self.last_board,move)
            self.q_last = self.getQ(self.last_board,move)
            return move
        else:
            Q_list = []
            for action in possible_moves:
                Q_list.append(self.getQ(self.last_board,action))
            maxQ = max(Q_list)
            
            if Q_list.count(maxQ) > 1:
                best_options = [i for i in range(len(possible_moves)) if Q_list[i] == maxQ]
                i = random.choice(best_options)
            else:
                i = Q_list.index(maxQ)
            self.state_action_last = (self.last_board, possible_moves[i])
            self.q_last = self.getQ(self.last_board, possible_moves[i])
            return possible_moves[i]
        
        
        
    def getQ(self, state, action):
        if(self.Q.get((state,action))) is None:
            self.Q[(state,action)] = 1.0
        return self.Q.get((state,action))
    
    
    
    
    def updateQ(self,reward, state, possible_moves):
        q_list = []
        for moves in possible_moves:
            q_list.append(self.getQ(tuple(state), moves))
        if q_list:
            max_q_next = max(q_list)
        else:
            max_q_next = 0.0
        self.Q[self.state_action_last] = self.q_last + self.alpha * ((reward + self.gamma*max_q_next) - self.q_last)
        
        
        
        
    def saveQtable(self, file_name):
        with open(file_name, 'wb') as handle:
            pickle.dump(self.Q, handle, protocol = pickle.HIGHEST_PROTOCOL)
            
    
    
    def loadQtable(self,file_name):
        with open(file_name, 'rb') as handle:
            self.Q = pickle.load(handle)
    
    
   

In [3]:
class TicTacToe:
    def __init__(self, training = False):
        self.board = [' ']*9
        
        self.done = False
        self.computer = None
        self.training = training
        self.player1 = None
        self.player2 = None
        
        
        
    def reset(self):
        if(self.training):
            self.board = [' ']*9
            return
        
    def show_board(self,board):
        print("  ")
        print(" %c | %c | %c "%(board[0],board[1],board[2]))
        print("-%c-|-%c-|-%c-"%('-','-','-'))
        
        print(" %c | %c | %c "%(board[3],board[4],board[5]))
        print("-%c-|-%c-|-%c-"%('-','-','-'))
        
        print(" %c | %c | %c "%(board[6],board[7],board[8]))
        print("  ")

        
        
    def evaluate(self,ch):
        #ROW CHECKING
        for i in range(3):
            if(ch == self.board[i*3] == self.board[i*3 + 1] and self.board[i * 3 + 1] == self.board[i *3 + 2]):
                return 1.0, True
        
        #COLUMN CHECKING
        for i in range(3):
            if(ch == self.board[i + 0] == self.board[i + 3] and self.board[i + 3] == self.board[i + 6]):
                return 1.0, True
        # DIAGONAL CHECKING
        if (ch == self.board[0] == self.board[4] and self.board[4] == self.board[8]):
            return 1.0, True

        if (ch == self.board[2] == self.board[4] and self.board[4] == self.board[6]):
            return 1.0, True
        # "if filled draw"
        if not any(c == ' ' for c in self.board):
            return 0.5, True

        return 0.0, False
                
    
    def possible_moves(self):
        return [moves + 1 for moves, v in enumerate(self.board) if v == ' ']
    
    
    
    def step(self, isX, move):
        if(isX):
            ch = 'X'
        else:
            ch = 'O'
        if(self.board[move-1] != ' '):
            return -5, True
        
        self.board[move-1] = ch
        reward, done = self.evaluate(ch)
        return reward, done
    
    
    
    
    def startTraining(self,player1, player2):
        if(isinstance(player1,QLearning) and isinstance(player2, QLearning)):
            self.training = True
            self.player1=player1
            self.player2=player2          
    
    
    
    
    def train(self, iterations):
        if(self.training):
            p1_won = 0
            p1_draw = 0
            p1_lose = 0
            p1_invalid = 0
            p2_won = 0
            p2_draw = 0
            p2_lose = 0
            p2_invalid = 0
            for i in range(iterations):
                #print("Training", i)
                self.player1.game_begin()
                self.player2.game_begin()
                self.reset()
                done = False
                isX = random.choice([True, False])
                while not done:
                    if isX:
                        move = self.player1.epsilon_greedy(self.board,self.possible_moves())
                    else:
                        move = self.player2.epsilon_greedy(self.board, self.possible_moves())
                        
                    reward, done = self.step(isX,move)
                    
                    if(reward == 1):
                        if(isX):
                            p1_won += 1
                            self.player1.updateQ(reward, self.board, self.possible_moves())
                            self.player2.updateQ(-1 * reward, self.board, self.possible_moves())
                        else:
                            p2_won += 1
                            self.player1.updateQ(-1 * reward, self.board, self.possible_moves())
                            self.player2.updateQ(reward, self.board, self.possible_moves())

                    elif (reward == 0.5):  # draw
                        p1_draw += 1
                        p2_draw += 1
                        self.player1.updateQ(reward, self.board, self.possible_moves())
                        self.player2.updateQ(reward, self.board, self.possible_moves())


                    elif (reward == -5):  # illegal move
                        if (isX):
                            p1_invalid += 1
                            self.player1.updateQ(reward, self.board, self.possible_moves())
                        else:
                            p2_invalid += 1
                            self.player2.updateQ(reward, self.board, self.possible_moves())

                    elif (reward == 0):
                        if (isX):  # update opposite
                            self.player2.updateQ(reward, self.board, self.possible_moves())
                        else:
                            self.player1.updateQ(reward, self.board, self.possible_moves())

                    isX = not isX  #
                #self.show_board(self.board)

        print(p1_won,p2_won,p1_draw,p2_draw,p1_invalid,p2_invalid)

    #save Qtables
    def saveStates(self):
        self.player1.saveQtable("player1states")
        self.player2.saveQtable("player2states")    
        
        
        #save Qtables
    def loadStates(self):
        self.computer.loadQtable("player1states")

        
        
    def playGame(self,computer):
        self.computer = computer
        self.loadStates()
        self.computer.game_begin()
        self.reset()
        done = False
        isX = random.choice([True, False])
        while not done:
            if isX:
                move = self.computer.epsilon_greedy(self.board,self.possible_moves())
                time.sleep(1)
            else:
                move = int(input("Enter Your Move: "))
                if move not in [1,2,3,4,5,6,7,8,9]:
                    print("Invalid!")
                    continue
            
            reward, done = self.step(isX,move)
            self.show_board(self.board)
            if reward == -5:
                print("invalid_step")
                done = False
                continue
            elif reward == 1:
                if isX:
                    print("Computer Won!")
                else:
                    print("You Won!")
            elif reward == 0.5:
                print("Match Drawn!")
            isX = not isX
            

            


In [4]:
game = TicTacToe(True) #game instance, True means training
'''player1= QLearning() #player1 learning agent
player2 =QLearning() #player2 learning agent
game.startTraining(player1,player2) #start training
game.train(200000) #train for 200,000 iterations
game.saveStates()  #save Qtable'''


'player1= QLearning() #player1 learning agent\nplayer2 =QLearning() #player2 learning agent\ngame.startTraining(player1,player2) #start training\ngame.train(200000) #train for 200,000 iterations\ngame.saveStates()  #save Qtable'

In [7]:
computer = QLearning(epsilon=0.2)
game.playGame(computer)

  
   |   |   
---|---|---
 X |   |   
---|---|---
   |   |   
  
Enter Your Move: 5
  
   |   |   
---|---|---
 X | O |   
---|---|---
   |   |   
  
  
 X |   |   
---|---|---
 X | O |   
---|---|---
   |   |   
  
Enter Your Move: 7
  
 X |   |   
---|---|---
 X | O |   
---|---|---
 O |   |   
  
  
 X | X |   
---|---|---
 X | O |   
---|---|---
 O |   |   
  
Enter Your Move: 3
  
 X | X | O 
---|---|---
 X | O |   
---|---|---
 O |   |   
  
You Won!


In [6]:
#Reference https://github.com/Rohithkvsp/Tic-Tac-Toe-Reinforcement-learning